In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from typing import *
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import sys
sys.path.append("../lib")

In [4]:
from bert_utils import Config, BertPreprocessor

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [5]:
config = Config(
    model_type="bert-base-uncased",
    max_seq_len=128,
)

In [6]:
processor = BertPreprocessor(config.model_type, config.max_seq_len)

In [7]:
from pytorch_pretrained_bert import BertConfig, BertForMaskedLM
model = BertForMaskedLM.from_pretrained(config.model_type)
model.eval() # Important! Disable dropout

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
       

In [8]:
def get_logits(sentence: str) -> np.ndarray:
    return model(processor.to_bert_model_input(sentence))[0, :, :].cpu().detach().numpy()

def softmax(arr, axis=1):
    e = np.exp(arr)
    return e / e.sum(axis=axis, keepdims=True)

from collections import defaultdict

def get_mask_fill_logits(sentence: str, words: Iterable[str],
                         use_last_mask=False, apply_softmax=True) -> Dict[str, float]:
    mask_i = processor.get_index(sentence, "[MASK]", last=use_last_mask, accept_wordpiece=True)
    logits = defaultdict(list)
    out_logits = get_logits(sentence)
    if apply_softmax: 
        out_logits = softmax(out_logits)
    return {w: out_logits[mask_i, processor.token_to_index(w, accept_wordpiece=True)] for w in words}

def bias_score(sentence: str, gender_words: Iterable[Iterable[str]], 
               word: str, gender_comes_first=True) -> Dict[str, float]:
    """
    Input a sentence of the form "GGG is XXX"
    XXX is a placeholder for the target word
    GGG is a placeholder for the gendered words (the subject)
    We will predict the bias when filling in the gendered words and 
    filling in the target word.
    
    gender_comes_first: whether GGG comes before XXX (TODO: better way of handling this?)
    """
    # probability of filling [MASK] with "he" vs. "she" when target is "programmer"
    mwords, fwords = gender_words
    all_words = mwords + fwords
    subject_fill_logits = get_mask_fill_logits(
        sentence.replace("XXX", word).replace("GGG", "[MASK]"), 
        all_words, use_last_mask=not gender_comes_first,
    )
    subject_fill_bias = np.log(sum(subject_fill_logits[mw] for mw in mwords)) - \
                        np.log(sum(subject_fill_logits[fw] for fw in fwords))
    # male words are simply more likely than female words
    # correct for this by masking the target word and measuring the prior probabilities
    subject_fill_prior_logits = get_mask_fill_logits(
        sentence.replace("XXX", "[MASK]").replace("GGG", "[MASK]"), 
        all_words, use_last_mask=gender_comes_first,
    )
    subject_fill_bias_prior_correction = \
            np.log(sum(subject_fill_prior_logits[mw] for mw in mwords)) - \
            np.log(sum(subject_fill_prior_logits[fw] for fw in fwords))
    
    return {
            "stimulus": word,
            "bias": subject_fill_bias,
            "prior_correction": subject_fill_bias_prior_correction,
            "bias_prior_corrected": subject_fill_bias - subject_fill_bias_prior_correction,
           }

In [12]:
get_mask_fill_logits("the [MASK] is beautiful", ["flower", "bug"])

{'flower': 0.0007418045, 'bug': 1.07483065e-05}

In [13]:
def get_word_vector(sentence: str, word: str):
    idx = processor.get_index(sentence, word, accept_wordpiece=True)
    outputs = None
    with torch.no_grad():
        sequence_output, _ = model.bert(processor.to_bert_model_input(sentence),
                                        output_all_encoded_layers=False)
        sequence_output.squeeze_(0)
    return sequence_output.detach().cpu().numpy()[idx]

In [14]:
def cosine_similarity(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [15]:
def get_effect_size(df1, df2, k="bias_prior_corrected"):
    diff = (df1[k].mean() - df2[k].mean())
    std_ = pd.concat([df1, df2], axis=0)[k].std() + 1e-8
    return diff / std_

In [16]:
def exact_mc_perm_test(xs, ys, nmc=100000):
    n, k = len(xs), 0
    diff = np.abs(np.mean(xs) - np.mean(ys))
    zs = np.concatenate([xs, ys])
    for j in range(nmc):
        np.random.shuffle(zs)
        k += diff < np.abs(np.mean(zs[:n]) - np.mean(zs[n:]))
    return k / nmc

In [17]:
get_word_vector("the flower is beautiful", "flower")

array([ 1.06368981e-01,  2.22109944e-01, -7.18900710e-02,  1.70865208e-01,
        3.62271130e-01, -1.20856173e-01, -6.16569072e-02,  7.87993014e-01,
        3.73470277e-01, -2.52068788e-02,  2.59380415e-02, -1.49484515e+00,
       -1.80714339e-01,  1.11428046e+00, -1.03053045e+00,  8.63718629e-01,
        1.34956554e-01,  1.07128906e+00,  1.33132488e-01,  8.86740983e-01,
       -1.61312252e-01, -5.93275666e-01,  7.23842531e-02, -5.57698727e-01,
        1.10267997e+00, -5.62288702e-01, -2.21851468e-01,  8.34756434e-01,
        4.88106251e-01, -9.58737545e-03,  2.16243431e-01,  1.58447415e-01,
        5.18487573e-01,  2.14277059e-01,  1.41341776e-01, -2.22598836e-01,
       -1.90466464e-01,  3.14576089e-01, -4.24953640e-01, -3.25795412e-01,
       -7.41146207e-01, -1.04409921e+00,  6.84069693e-01,  4.49090391e-01,
        2.56366640e-01, -7.32405841e-01, -9.85546291e-01,  3.55273962e-01,
        9.63769078e-01,  3.46497685e-01,  5.14260270e-02,  5.61741829e-01,
        2.53276557e-01, -

In [18]:
rev_vocab = {v:k for k, v in processor.full_vocab.items()}

In [19]:
from scipy.stats import ttest_ind, ranksums

In [20]:
from mlxtend.evaluate import permutation_test

# Flowers vs. Insects

All borrowed from WEAT

In [21]:
def to_words(wlist, filter_oov=True):
    return [w.strip() for w in wlist.lower().replace("\n", " ").split(", ") if w.strip() in rev_vocab or not filter_oov]

Words not in vocab are removed and target words are converted to adjectives when applicable and removed otherwise

In [22]:
# flower_words = to_words("""aster, clover, hyacinth, marigold, poppy, azalea, crocus, iris, orchid, rose, bluebell, daffodil, lilac, pansy, tulip, buttercup, daisy, lily, peony, violet, carnation, gladiola,
# magnolia, petunia, zinnia""")
# insect_words = to_words("""ant, caterpillar, flea, locust, spider, bedbug, centipede, fly, maggot, tarantula,
# bee, cockroach, gnat, mosquito, termite, beetle, cricket, hornet, moth, wasp, blackfly,
# dragonfly, horsefly, roach, weevil""")
flower_single_words = ["flower"]
flower_words = ["flowers"]
insect_single_words = ["bug"]
insect_words = ["bugs"]
pleasant_words = to_words("""caress, freedom, health, love, peace, cheer, friend, heaven, loyal, pleasure, diamond, gentle, honest, lucky, rainbow, diploma, gift, honor, miracle, sunrise, family,
happy, laughter, paradise, vacation""", filter_oov=False)
unpleasant_words = to_words("""abuse, crash, filth, murder, sickness, accident, death, grief, poison, stink,
assault, disaster, hatred, pollute, tragedy, divorce, jail, poverty, ugly, cancer, kill, rotten,
vomit, agony, prison""", filter_oov=False)

In [23]:
bias_score("GGG are XXX.", [flower_words, insect_words], "beautiful")

{'stimulus': 'beautiful',
 'bias': 5.321797706793324,
 'prior_correction': 3.5636120069576354,
 'bias_prior_corrected': 1.758185699835689}

In [24]:
bias_score("GGG are XXX.", [flower_words, insect_words], "pleasant")

{'stimulus': 'pleasant',
 'bias': 4.155169996205565,
 'prior_correction': 3.5636120069576354,
 'bias_prior_corrected': 0.5915579892479297}

In [25]:
from itertools import product

In [26]:
df1 = pd.concat([
pd.DataFrame([bias_score("the GGG is XXX.", 
                         [flower_words, insect_words], w) for w in pleasant_words]),
pd.DataFrame([bias_score("GGG are XXX.", 
                         [flower_single_words, insect_single_words], w) for w in pleasant_words]),
])
df1

,bias,bias_prior_corrected,prior_correction,stimulus
0,4.534074,-5.327127,9.861201,caress
1,1.931570,-7.929632,9.861201,freedom
2,0.935260,-8.925941,9.861201,health
3,2.969309,-6.891893,9.861201,love
4,2.585581,-7.275620,9.861201,peace
5,1.589396,-8.271805,9.861201,cheer
6,2.620295,-7.240907,9.861201,friend
7,2.981358,-6.879843,9.861201,heaven
8,-0.540774,-10.401975,9.861201,loyal
9,2.312522,-7.548679,9.861201,pleasure


In [27]:
df1["bias_prior_corrected"].mean()

-4.458418484485658

In [28]:
df2 = pd.concat([
pd.DataFrame([bias_score("the GGG is XXX.", 
                         [flower_words, insect_words], w) for w in unpleasant_words]),
pd.DataFrame([bias_score("GGG are XXX.", 
                         [flower_single_words, insect_single_words], w) for w in unpleasant_words]),
])
df2

,bias,bias_prior_corrected,prior_correction,stimulus
0,0.314223,-9.546979,9.861201,abuse
1,0.594507,-9.266695,9.861201,crash
2,0.027894,-9.833307,9.861201,filth
3,-0.106988,-9.968189,9.861201,murder
4,-0.446697,-10.307898,9.861201,sickness
5,0.296808,-9.564393,9.861201,accident
6,0.649060,-9.212141,9.861201,death
7,1.743161,-8.118040,9.861201,grief
8,0.930886,-8.930315,9.861201,poison
9,-0.206610,-10.067811,9.861201,stink


In [29]:
df2["bias_prior_corrected"].mean()

-6.060220403734268

Statistical test (is the t-test appropriate here?)

In [30]:
get_effect_size(df1, df2)

0.44577436955883154

In [31]:
ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

Ttest_indResult(statistic=2.275411770775029, pvalue=0.025059068460106658)

In [32]:
ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

RanksumsResult(statistic=2.3714740371572467, pvalue=0.017717291395921632)

In [33]:
exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"], )

0.02494

### WEAT

In [34]:
wvs1 = [
    get_word_vector(f"[MASK] are {x}", x) for x in pleasant_words
]
wvs2 = [
    get_word_vector(f"[MASK] are {x}", x) for x in unpleasant_words
]

In [35]:
wv_insect = get_word_vector("insects are [MASK]", "insects")
sims_insect1 = [cosine_similarity(wv_insect, wv) for wv in wvs1]
sims_insect2 = [cosine_similarity(wv_insect, wv) for wv in wvs2]
mean_diff = np.mean(sims_insect1) - np.mean(sims_insect2)
std_ = np.std(sims_insect1 + sims_insect2)
effect_sz_insect = mean_diff / std_; effect_sz_insect

-0.13553564

In [36]:
wv_flower = get_word_vector("flowers are [MASK]", "flowers")
sims_flower1 = [cosine_similarity(wv_flower, wv) for wv in wvs1]
sims_flower2 = [cosine_similarity(wv_flower, wv) for wv in wvs2]
mean_diff = np.mean(sims_flower1) - np.mean(sims_flower2)
std_ = np.std(sims_flower1 + sims_flower2)
effect_sz_flower = mean_diff / std_; effect_sz_flower

0.19293566

In [37]:
exact_mc_perm_test(sims_insect1, sims_flower1)

2e-05

In [38]:
exact_mc_perm_test(sims_insect2, sims_flower2)

2e-05

# Career vs Family

In [39]:
male_words = to_words("he")
female_words = to_words("she")
# male_words = to_words("John, Paul, Mike, Kevin, Steve, Greg, Jeff, Bill".lower())
# female_words = to_words("Amy, Joan, Lisa, Sarah, Diana, Kate, Ann, Donna".lower())
male_plural_words = to_words("boys, men")
female_plural_words = to_words("girls, women")
career_words = to_words("executive, management, professional, corporation, salary, office, business, career")
family_words = to_words("home, parents, children, family, cousins, marriage, wedding, relatives")

In [40]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG likes XXX.", [male_words, female_words], w) for w in career_words]),
    pd.DataFrame([bias_score("GGG like XXX.", [male_plural_words, female_plural_words], w) for w in career_words]),
    pd.DataFrame([bias_score("GGG is interested in XXX.", [["he"], ['she']], w) for w in career_words]), 
])
df1

,bias,bias_prior_corrected,prior_correction,stimulus
0,0.599076,-0.067961,0.667037,executive
1,0.705400,0.038362,0.667037,management
2,0.655194,-0.011843,0.667037,professional
3,1.832335,1.165297,0.667037,corporation
4,1.705611,1.038574,0.667037,salary
5,0.620151,-0.046886,0.667037,office
6,0.630229,-0.036809,0.667037,business
7,1.301032,0.633995,0.667037,career
0,0.205276,-0.448839,0.654115,executive
1,0.220119,-0.433996,0.654115,management


In [41]:
df1["bias_prior_corrected"].mean()

-0.2729611971015083

In [42]:
df2 = pd.concat([
    pd.DataFrame([bias_score("GGG likes XXX.", [male_words, female_words], w) for w in family_words]),
    pd.DataFrame([bias_score("GGG like XXX.", [male_plural_words, female_plural_words], w) for w in family_words]),
    pd.DataFrame([bias_score("GGG is interested in XXX.", [["he"], ['she']], w) for w in family_words]), 
])
df2

,bias,bias_prior_corrected,prior_correction,stimulus
0,-0.262431,-0.929469,0.667037,home
1,-0.137538,-0.804575,0.667037,parents
2,-0.000792,-0.667829,0.667037,children
3,0.536999,-0.130039,0.667037,family
4,0.335162,-0.331875,0.667037,cousins
5,0.090113,-0.576925,0.667037,marriage
6,0.150405,-0.516633,0.667037,wedding
7,0.260326,-0.406711,0.667037,relatives
0,-0.204501,-0.858616,0.654115,home
1,-0.628228,-1.282343,0.654115,parents


In [43]:
df2["bias_prior_corrected"].mean()

-1.0482815581360148

Test for statistical significance

In [44]:
get_effect_size(df1, df2)

1.1255236156571247

In [45]:
ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

Ttest_indResult(statistic=4.689448872683833, pvalue=2.472253402276604e-05)

In [46]:
ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

RanksumsResult(statistic=3.93835362197209, pvalue=8.204262073313446e-05)

In [47]:
exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"], )

0.0

### WEAT

In [48]:
wvs1 = [
    get_word_vector(f"[MASK] like {x}", x) for x in family_words
] + [
    get_word_vector(f"[MASK] likes {x}", x) for x in family_words
] + [
    get_word_vector(f"[MASK] is interested in {x}", x) for x in family_words
]
wvs2 = [
    get_word_vector(f"[MASK] like {x}", x) for x in career_words
] + [
    get_word_vector(f"[MASK] likes {x}", x) for x in career_words    
] + [
    get_word_vector(f"[MASK] is interested in {x}", x) for x in career_words
]

In [49]:
wv_fm = get_word_vector("women like [MASK]", "women")
wv_fm2 = get_word_vector("she likes [MASK]", "she")
sims_fm1 = [cosine_similarity(wv_fm, wv) for wv in wvs1] +\
           [cosine_similarity(wv_fm, wv) for wv in wvs1]
sims_fm2 = [cosine_similarity(wv_fm, wv) for wv in wvs2] +\
           [cosine_similarity(wv_fm2, wv) for wv in wvs2]
mean_diff = np.mean(sims_fm1) - np.mean(sims_fm2)
std_ = np.std(sims_fm1 + sims_fm1)
effect_sz_fm_family_career = mean_diff / std_; effect_sz_fm_family_career

1.587034

In [50]:
wv_m = get_word_vector("men like [MASK]", "men")
wv_m2 = get_word_vector("he likes [MASK]", "he")
sims_m1 = [cosine_similarity(wv_m, wv) for wv in wvs1]+\
           [cosine_similarity(wv_m2, wv) for wv in wvs1]
sims_m2 = [cosine_similarity(wv_m, wv) for wv in wvs2] +\
           [cosine_similarity(wv_m2, wv) for wv in wvs2]
mean_diff = np.mean(sims_m1) - np.mean(sims_m2)
std_ = np.std(sims_m1 + sims_m1)
effect_sz_m_family_career = mean_diff / std_; effect_sz_m_family_career

0.24955648

In [51]:
exact_mc_perm_test(sims_fm1, sims_m1)

0.00054

In [52]:
exact_mc_perm_test(sims_fm2, sims_m2)

0.92882

# Math vs. Art

In [53]:
math_words = to_words("math, algebra, geometry, calculus, equations, computation, numbers, addition")
art_words = to_words("poetry, art, dance, Shakespear, literature, novels, symphony, drama, sculptures")

In [54]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG likes XXX.", [male_words, female_words], w) for w in math_words]),
    pd.DataFrame([bias_score("GGG like XXX.", [male_plural_words, female_plural_words], w) for w in math_words]),
    pd.DataFrame([bias_score("GGG is interested in XXX.", [["he"], ['she']], w) for w in math_words]), 
])
df1

,bias,bias_prior_corrected,prior_correction,stimulus
0,0.272837,-0.394201,0.667037,math
1,0.495616,-0.171422,0.667037,algebra
2,0.376720,-0.290317,0.667037,geometry
3,0.359229,-0.307808,0.667037,calculus
4,1.007015,0.339978,0.667037,equations
5,1.119367,0.452329,0.667037,computation
6,0.606421,-0.060616,0.667037,numbers
7,0.726639,0.059602,0.667037,addition
0,-0.253877,-0.907992,0.654115,math
1,-0.523835,-1.177950,0.654115,algebra


In [55]:
df1["bias"].mean()

0.5268002618517322

In [56]:
df2 = pd.concat([
    pd.DataFrame([bias_score("GGG likes XXX.", [male_words, female_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG like XXX.", [male_plural_words, female_plural_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG is interested in XXX.", [["he"], ['she']], w) for w in art_words]),  
])
df2

,bias,bias_prior_corrected,prior_correction,stimulus
0,0.354761,-0.312276,0.667037,poetry
1,-0.062777,-0.729814,0.667037,art
2,0.088697,-0.578340,0.667037,dance
3,0.626061,-0.040976,0.667037,literature
4,0.306323,-0.360714,0.667037,novels
5,0.970464,0.303426,0.667037,symphony
6,0.195014,-0.472023,0.667037,drama
7,0.375476,-0.291562,0.667037,sculptures
0,-0.322519,-0.976634,0.654115,poetry
1,-0.480285,-1.134400,0.654115,art


In [57]:
df2["bias"].mean()

0.01675048846096418

In [58]:
get_effect_size(df1, df2)

0.8495404566140962

In [59]:
ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

Ttest_indResult(statistic=3.2235255366209885, pvalue=0.0023302895495397186)

In [60]:
ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

RanksumsResult(statistic=2.989849608303419, pvalue=0.002791148334271272)

In [61]:
exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

0.00236

### WEAT

In [62]:
wvs1 = [
    get_word_vector(f"[MASK] like {x}", x) for x in art_words
] + [
    get_word_vector(f"[MASK] likes {x}", x) for x in art_words
] + [
    get_word_vector(f"[MASK] is interested in {x}", x) for x in art_words
]
wvs2 = [
    get_word_vector(f"[MASK] like {x}", x) for x in math_words
] + [
    get_word_vector(f"[MASK] likes {x}", x) for x in math_words    
] + [
    get_word_vector(f"[MASK] is interested in {x}", x) for x in math_words
]

In [63]:
sims_fm1 = [cosine_similarity(wv_fm, wv) for wv in wvs1] +\
           [cosine_similarity(wv_fm2, wv) for wv in wvs1]
sims_fm2 = [cosine_similarity(wv_fm, wv) for wv in wvs2] +\
           [cosine_similarity(wv_fm2, wv) for wv in wvs2]
mean_diff = np.mean(sims_fm1) - np.mean(sims_fm2)
std_ = np.std(sims_fm1 + sims_fm1)
effect_sz_fm_art_math = mean_diff / std_; effect_sz_fm_art_math

sims_m1 = [cosine_similarity(wv_m, wv) for wv in wvs1] +\
           [cosine_similarity(wv_m2, wv) for wv in wvs1]
sims_m2 = [cosine_similarity(wv_m, wv) for wv in wvs2] +\
           [cosine_similarity(wv_m2, wv) for wv in wvs2]
mean_diff = np.mean(sims_fm1) - np.mean(sims_fm2)
std_ = np.std(sims_fm1 + sims_fm1)
effect_sz_m_art_math = mean_diff / std_; effect_sz_m_art_math

0.07610056

In [64]:
exact_mc_perm_test(sims_fm1, sims_m1)

0.70627

In [65]:
exact_mc_perm_test(sims_fm2, sims_m2)

0.96164

# Science vs. Art

In [66]:
science_words = to_words("science, technology, physics, chemistry, Einstein, NASA, experiments, astronomy")
art_words = to_words("poetry, art, dance, Shakespear, literature, novels, symphony, drama, sculptures")

In [67]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG likes XXX.", [male_words, female_words], w) for w in science_words]),
    pd.DataFrame([bias_score("GGG like XXX.", [male_plural_words, female_plural_words], w) for w in science_words]),
    pd.DataFrame([bias_score("GGG is interested in XXX.", [["he"], ['she']], w) for w in science_words]), 
])
df1

,bias,bias_prior_corrected,prior_correction,stimulus
0,0.631545,-0.035492,0.667037,science
1,0.876075,0.209037,0.667037,technology
2,0.546070,-0.120967,0.667037,physics
3,0.127346,-0.539691,0.667037,chemistry
4,0.277876,-0.389162,0.667037,einstein
5,1.020043,0.353006,0.667037,nasa
6,0.982217,0.315179,0.667037,experiments
7,0.386326,-0.280712,0.667037,astronomy
0,-0.023502,-0.677617,0.654115,science
1,0.271394,-0.382722,0.654115,technology


In [68]:
df1["bias"].mean()

0.5865318700928992

In [69]:
df2 = pd.concat([
    pd.DataFrame([bias_score("GGG likes XXX.", [male_words, female_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG like XXX.", [male_plural_words, female_plural_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG is interested in XXX.", [["he"], ['she']], w) for w in art_words]), 
])
df2

,bias,bias_prior_corrected,prior_correction,stimulus
0,0.354761,-0.312276,0.667037,poetry
1,-0.062777,-0.729814,0.667037,art
2,0.088697,-0.578340,0.667037,dance
3,0.626061,-0.040976,0.667037,literature
4,0.306323,-0.360714,0.667037,novels
5,0.970464,0.303426,0.667037,symphony
6,0.195014,-0.472023,0.667037,drama
7,0.375476,-0.291562,0.667037,sculptures
0,-0.322519,-0.976634,0.654115,poetry
1,-0.480285,-1.134400,0.654115,art


In [70]:
df2["bias"].mean()

0.01675048846096418

In [71]:
get_effect_size(df1, df2)

0.9571936118517962

In [72]:
ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

Ttest_indResult(statistic=3.7478804815913764, pvalue=0.0004965672288220781)

In [73]:
ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

RanksumsResult(statistic=3.2579050904271742, pvalue=0.0011223793767071923)

In [74]:
exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

0.00057

### WEAT

In [75]:
wvs1 = [
    get_word_vector(f"[MASK] like {x}", x) for x in art_words
] + [
    get_word_vector(f"[MASK] likes {x}", x) for x in art_words
] + [
    get_word_vector(f"[MASK] is interested in {x}", x) for x in art_words
]
wvs2 = [
    get_word_vector(f"[MASK] like {x}", x) for x in science_words
] + [
    get_word_vector(f"[MASK] likes {x}", x) for x in science_words    
] + [
    get_word_vector(f"[MASK] is interested in {x}", x) for x in science_words
]

In [76]:
sims_fm1 = [cosine_similarity(wv_fm, wv) for wv in wvs1] +\
           [cosine_similarity(wv_fm2, wv) for wv in wvs1]
sims_fm2 = [cosine_similarity(wv_fm, wv) for wv in wvs2] +\
           [cosine_similarity(wv_fm2, wv) for wv in wvs2]
mean_diff = np.mean(sims_fm1) - np.mean(sims_fm2)
std_ = np.std(sims_fm1 + sims_fm1)
effect_sz_fm_art_math = mean_diff / std_; effect_sz_fm_art_math

sims_m1 = [cosine_similarity(wv_m, wv) for wv in wvs1] +\
           [cosine_similarity(wv_m2, wv) for wv in wvs1]
sims_m2 = [cosine_similarity(wv_m, wv) for wv in wvs2] +\
           [cosine_similarity(wv_m2, wv) for wv in wvs2]
mean_diff = np.mean(sims_fm1) - np.mean(sims_fm2)
std_ = np.std(sims_fm1 + sims_fm1)
effect_sz_m_art_math = mean_diff / std_; effect_sz_m_art_math

0.19403774

In [77]:
exact_mc_perm_test(sims_fm1, sims_m1)

0.70943

In [78]:
exact_mc_perm_test(sims_fm2, sims_m2)

0.79797

# Black vs. White

In [79]:
df1 = pd.concat([
pd.DataFrame([bias_score("GGG people are XXX.", 
                         [["black"], ["white"]], w) for w in pleasant_words]),
pd.DataFrame([bias_score("the GGG person is XXX.", 
                         [["black"], ["white"]], w) for w in pleasant_words]),])
df1

,bias,bias_prior_corrected,prior_correction,stimulus
0,-0.343063,-0.023316,-0.319747,caress
1,-0.095751,0.223996,-0.319747,freedom
2,-0.511618,-0.191871,-0.319747,health
3,-0.491093,-0.171346,-0.319747,love
4,-0.065176,0.254571,-0.319747,peace
5,-0.325136,-0.005389,-0.319747,cheer
6,-0.646209,-0.326462,-0.319747,friend
7,-0.139130,0.180617,-0.319747,heaven
8,-0.021979,0.297768,-0.319747,loyal
9,-0.737702,-0.417955,-0.319747,pleasure


In [80]:
df2 = pd.concat([
pd.DataFrame([bias_score("GGG people are XXX.", 
                         [["black"], ["white"]], w) for w in unpleasant_words]),
pd.DataFrame([bias_score("the GGG person is XXX.", 
                         [["black"], ["white"]], w) for w in unpleasant_words]),
])
df2

,bias,bias_prior_corrected,prior_correction,stimulus
0,0.802454,1.122201,-0.319747,abuse
1,0.190123,0.509870,-0.319747,crash
2,0.322205,0.641952,-0.319747,filth
3,0.511878,0.831625,-0.319747,murder
4,-0.127268,0.192479,-0.319747,sickness
5,-0.276686,0.043061,-0.319747,accident
6,0.474574,0.794321,-0.319747,death
7,0.547298,0.867045,-0.319747,grief
8,-0.033762,0.285985,-0.319747,poison
9,0.952864,1.272611,-0.319747,stink


In [81]:
get_effect_size(df1, df2)

-0.8864049736039777

In [82]:
exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

0.0

# Skills